In [ ]:
# /**********************************************************************************************************

# File Name		: 	twitter_load.ipynb
# Purpose			:   Parsing tweets and loading into bigquery table
# Author			:   DeepSphere.AI, Inc.
# Date and Time 	: 	03/16/2021 10:30 hrs
# Version			: 	1.0 

# /************************************************************************************************************


def read_data():
    
    from google.cloud import bigquery
    import json
    import pandas as pd
    import numpy as np
    vAR_client = bigquery.Client()
    vAR_sql = "SELECT tweet FROM `vast-verve-292018.ds_ai.ds_ai_stream_tweets` limit 1000"
    vAR_df = vAR_client.query(vAR_sql).to_dataframe()
    vAR_lang_np = []
    vAR_text_np = []
    vAR_cust_np = []
    vAR_loc_np = []
    vAR_profile_image = []
    vAR_age_np = []
    vAR_gender_np = []
    vAR_summarize_np = []
    vAR_data_frame = pd.DataFrame()
    for lCount_idx in vAR_df.index:
        vAR_x = vAR_df['tweet'][lCount_idx]
        vAR_data = json.loads(vAR_x)
        vAR_cust_np.append(vAR_data['user']['screen_name'])
        vAR_lang_np.append(vAR_data['lang'])
        vAR_text_np.append(vAR_data['text'])
        vAR_loc_np.append(vAR_data['user']['location'])
        vAR_profile_image.append(vAR_data['user']['profile_image_url_https'])
        vAR_summarize_np.append(summarize_text(vAR_data['text']))
        vAR_download_image_path = "/home/dsailabusr1/PRAKASH/DE-Framework/DE-Framework-Impl/Framework/test.jpg"
        download_image(vAR_data['user']['profile_image_url_https'],vAR_download_image_path)
        vAR_age_,vAR_gender_ = age_and_gender(vAR_download_image_path)
        if len(vAR_age_)>0:
            vAR_age_ = vAR_age_[1:-1]
            vAR_age_mean = vAR_age_.split("-")
            vAR_val = (int(vAR_age_mean[0])+int(vAR_age_mean[1]))/2
            vAR_age_np.append(str(int(vAR_val)))
        else:
            vAR_age_np.append(vAR_age_)
        vAR_gender_np.append(vAR_gender_)
    vAR_data_frame['customer_id'] = vAR_cust_np
    vAR_data_frame['language'] = vAR_lang_np
    vAR_data_frame['text_content'] = vAR_text_np
    vAR_data_frame['location'] = vAR_loc_np
    vAR_data_frame['profile_image_url'] = vAR_profile_image
    vAR_data_frame['age'] = vAR_age_np
    vAR_data_frame['gender'] = vAR_gender_np
    vAR_data_frame['summary_or_insights'] = vAR_summarize_np
    
    vAR_data_frame = vAR_data_frame[vAR_data_frame['location'].notnull()]
    return vAR_data_frame

In [ ]:
import nltk
import heapq
import re
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
!sudo chmod -R 777 '/home/dsailabusr1/PRAKASH/DE-Framework/DE-Framework-Impl/Framework/test.jpg'

In [ ]:
#Download profile image
def download_image(url,path):
    import requests
    vAR_r = requests.get(url)
    with open(path, "wb") as vAR_f:
        vAR_f.write(vAR_r.content)

In [ ]:
# # This is for age and gender detecion
# # Downloading pretrained data and unzipping it
# !gdown https://drive.google.com/uc?id=1_aDScOvBeBLCn_iv0oxSO8X1ySQpSbIS
# # https://drive.google.com/uc?id=1_aDScOvBeBLCn_iv0oxSO8X1ySQpSbIS
# !unzip /home/dsailabusr1/PRAKASH/DE-Framework/DE-Framework-Impl/Framework/modelNweight.zip

In [ ]:
class age_detect:
    # Import required modules
    import cv2 as cv
    import math
    import time
    # from google.colab.patches import cv2_imshow
    vAR_faceProto = "modelNweight/opencv_face_detector.pbtxt"
    vAR_faceModel = "modelNweight/opencv_face_detector_uint8.pb"

    vAR_ageProto = "modelNweight/age_deploy.prototxt"
    vAR_ageModel = "modelNweight/age_net.caffemodel"

    vAR_genderProto = "modelNweight/gender_deploy.prototxt"
    vAR_genderModel = "modelNweight/gender_net.caffemodel"

    vAR_MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
    vAR_ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
    vAR_vAR_genderList = ['Male', 'Female']

    # Load network
    vAR_ageNet = cv.dnn.readNet(vAR_ageModel, vAR_ageProto)
    vAR_genderNet = cv.dnn.readNet(vAR_genderModel, vAR_genderProto)
    vAR_faceNet = cv.dnn.readNet(vAR_faceModel, vAR_faceProto)

    vAR_padding = 20

    def getFaceBox(self,net, frame, conf_threshold=0.7):
        import cv2 as cv
        vAR_frameOpencvDnn = frame.copy()
        vAR_frameHeight = vAR_frameOpencvDnn.shape[0]
        vAR_frameWidth = vAR_frameOpencvDnn.shape[1]
        vAR_blob = cv.dnn.blobFromImage(vAR_frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

        net.setInput(vAR_blob)
        vAR_detections = net.forward()
        vAR_bboxes = []
        for i in range(vAR_detections.shape[2]):
            vAR_confidence = vAR_detections[0, 0, i, 2]
            if vAR_confidence > conf_threshold:
                vAR_x1 = int(vAR_detections[0, 0, i, 3] * vAR_frameWidth)
                vAR_y1 = int(vAR_detections[0, 0, i, 4] * vAR_frameHeight)
                vAR_x2 = int(vAR_detections[0, 0, i, 5] * vAR_frameWidth)
                vAR_y2 = int(vAR_detections[0, 0, i, 6] * vAR_frameHeight)
                vAR_bboxes.append([vAR_x1, vAR_y1, vAR_x2, vAR_y2])
                cv.rectangle(vAR_frameOpencvDnn, (vAR_x1, vAR_y1), (vAR_x2, vAR_y2), (0, 255, 0), int(round(vAR_frameHeight/150)), 8)
        return vAR_frameOpencvDnn, vAR_bboxes

    def age_gender_detector(self,frame):
        # Read frame
        import time
        import cv2 as cv
        t = time.time()
        vAR_frameFace, vAR_bboxes = self.getFaceBox(self.vAR_faceNet, frame)
        vAR_age=''
        vAR_gender = ''
        for bbox in vAR_bboxes:
            # print(bbox)
            face = frame[max(0,bbox[1]-self.vAR_padding):min(bbox[3]+self.vAR_padding,frame.shape[0]-1),max(0,bbox[0]-self.vAR_padding):min(bbox[2]+self.vAR_padding, frame.shape[1]-1)]
            vAR_blob = cv.dnn.blobFromImage(face, 1.0, (227, 227), self.vAR_MODEL_MEAN_VALUES, swapRB=False)
            self.vAR_genderNet.setInput(vAR_blob)
            vAR_genderPreds = self.vAR_genderNet.forward()
            vAR_gender = self.vAR_vAR_genderList[vAR_genderPreds[0].argmax()]
            self.vAR_ageNet.setInput(vAR_blob)
            vAR_agePreds = self.vAR_ageNet.forward()
            vAR_age = self.vAR_ageList[vAR_agePreds[0].argmax()]

    #         label = "{},{}".format(vAR_gender, vAR_age)
    #         print(label)
    #         cv.putText(vAR_frameFace, label, (bbox[0], bbox[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv.LINE_AA)
        return vAR_age,vAR_gender

In [ ]:
def age_and_gender(path):
    import cv2 as cv
    vAR_age_ = ""
    vAR_gender_ = ""
    try:
        input = cv.imread(path)
        vAR_resize = cv.resize(input, (227,227))
        rName_Obj = age_detect()
        vAR_age_,vAR_gender_ = rName_Obj.age_gender_detector(vAR_resize)
    except Exception as e:
        pass
    return vAR_age_,vAR_gender_

In [ ]:
def twitter_load():
    from google.cloud import bigquery
    import datetime
    import time
    vAR_client = bigquery.Client()
    vAR_df = read_data()
    vAR_topic = 'covid'
    vAR_df['platform_source'] = 'twitter'
    vAR_df['topic_or_hashtag'] = vAR_topic
    vAR_df['age'] = vAR_df.age.astype(str)
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
    vAR_df['created_time'] = st
    insert_flag = insert_bigquery(bigquery,vAR_client,vAR_df)
    return insert_flag
#     update_flag = update_twitter_summary(client,df)

In [ ]:
def insert_bigquery(bigquery,client,df):

    import datetime
    import pandas
    import pytz
    # TODO(developer): Set table_id to the ID of the table to create.
    vAR_table_id = "vast-verve-292018.ds_ai.ds_ai_twitter_data"
    vAR_job_config = bigquery.LoadJobConfig(
        # Specify a (partial) schema. All columns are always written to the
        # table. The schema is used to assist in data type definitions.
        # Optionally, set the write disposition. BigQuery appends loaded rows
        # to an existing table by default, but with WRITE_TRUNCATE write
        # disposition it replaces the table with the loaded data.
        write_disposition="WRITE_APPEND",
    )

    vAR_job = client.load_table_from_dataframe(
        df, vAR_table_id, job_config=vAR_job_config
    )  # Make an API request.
    vAR_job.result()  # Wait for the job to complete.

    vAR_table = client.get_table(vAR_table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            vAR_table.num_rows, len(vAR_table.schema), vAR_table_id
        )
    )
    return True

In [ ]:
# https://stackabuse.com/text-summarization-with-nltk-in-python/
def summarize_text(article_text):
    import nltk
    import heapq
    import re
    # Removing Square Brackets and Extra Spaces
    article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
    article_text = re.sub(r'\s+', ' ', article_text)
    # Removing special characters and digits
    vAR_formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
    vAR_formatted_article_text = re.sub(r'\s+', ' ', vAR_formatted_article_text)
    # Converting Text To Sentences
    vAR_sentence_list = nltk.sent_tokenize(article_text)
    vAR_stopwords = nltk.corpus.stopwords.words('english')
    # Find Weighted Frequency of Occurrence
    vAR_word_frequencies = {}
    for lCount_word in nltk.word_tokenize(vAR_formatted_article_text):
        if lCount_word not in vAR_stopwords:
            if lCount_word not in vAR_word_frequencies.keys():
                vAR_word_frequencies[lCount_word] = 1
            else:
                vAR_word_frequencies[lCount_word] += 1
    vAR_maximum_frequncy = max(vAR_word_frequencies.values())

    for lCount_word in vAR_word_frequencies.keys():
        vAR_word_frequencies[lCount_word] = (vAR_word_frequencies[lCount_word]/vAR_maximum_frequncy)
    # Calculating Sentence Scores
    vAR_sentence_scores = {}
    for sent in vAR_sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in vAR_word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in vAR_sentence_scores.keys():
                        vAR_sentence_scores[sent] = vAR_word_frequencies[word]
                    else:
                        vAR_sentence_scores[sent] += vAR_word_frequencies[word]
    vAR_summary_sentences = heapq.nlargest(4, vAR_sentence_scores, key=vAR_sentence_scores.get)
    vAR_summary = ' '.join(vAR_summary_sentences)
    return vAR_summary

In [ ]:
# /**********************************************************************************************************
# Disclaimer.

# We are providing this code block strictly for learning and researching, this is not a production ready code.
# We have no liability on this particular code under any circumstances; users should
# use this code on their own risk. All software,hardware and other products that are
# referenced in these materials belong to the respective vendor who developed or who owns 
# this product.
# /**********************************************************************************************************
